# Question

## Q1: How to implement `min()` and `max()` aggregation? (TPCH-2, 15)

## Q2: How to implement `LIKE` (TPCH-2, 9, 13, 14, 16, 20)

## Q3: How to implement `EXTRACT()` (TPCH-7, 8, 9)

## Q4: How to implement `SUBSTRING()` (TPCH-22)

# FINISHED

## TPCH - 1 ✅

## TPCH - 3 ✅

## TPCH - 4 ✅

## TPCH - 5 ✅

## TPCH - 6 ✅

## TPCH - 10 ✅

## TPCH - 19 ✅

# Next Step

## TPCH - 11
## TPCH - 12
## TPCH - 17
## TPCH - 18
## TPCH - 19
## TPCH - 21

# Example


## Create a Relation

A relation R can be created using `pysdql.Relation(name='R')`.

In [1]:
import pysdql

r = pysdql.Relation(name='R')

print(r)

R


## Iteration

The iteration environment is automatically generated for relations in the construction method `__init__`, which can be accessed using `R.iter_expr`. By default, the iteration expression is presented as a key-value pair. 

In [2]:
r.iter_expr

sum (<r_k, r_v> in R)

## Select Columns

Columns of relation R can be selected using `R[['col1', 'col2', ..., 'coln']]`. This is implemented by overloading `__getitem__` method.

In [3]:
cols_a = r[['a']]

sum (<r_k, r_v> in R) { < a=r_k.a > }


In [4]:
cols_bc = r[['b', 'c']]

sum (<r_k, r_v> in R) { < b=r_k.b, c=r_k.c > }


## ColUnit

ColUnit is designed for creating conditional expressions by performing arithmetic operations ( +, -, *, / ) and logical operations ( &, | ). A ColUnit object can be created by `R['col']` or `R.col`, where `R` is a relation object and `col` is the column name. This is implemented by overloading `__getitem__` and `__getattr__` methods.

In [5]:
col_a = r['a']

col_a, type(col_a)

(r_k.a, pysdql.core.dtypes.ColumnUnit.ColUnit)

In [6]:
col_b = r.b

col_b, type(col_b)

(r_k.b, pysdql.core.dtypes.ColumnUnit.ColUnit)

## Conditional

Considering tpch-19, the SQL expression can be translated to the following conditial expression.

`p_partkey = l_partkey
and p_brand = ':1'
and p_container in ('SM CASE', 'SM BOX', 'SM PACK', 'SM PKG')
and l_quantity >= :4 and l_quantity <= :4 + 10
and p_size between 1 and 5
and l_shipmode in ('AIR', 'AIR REG')
and l_shipinstruct = 'DELIVER IN PERSON'`

In [7]:
lineitem = pysdql.Relation(name='lineitem', cols=pysdql.LINEITEM_COLS)
part = pysdql.Relation(name='part', cols=pysdql.PART_COLS)

c1 = (part['p_partkey'] == lineitem['l_partkey']) \
     & (part['p_brand'] == ':1') \
     & (part['p_container'].isin(('SM CASE', 'SM BOX', 'SM PACK', 'SM PKG'))) \
     & (lineitem['l_quantity'] >= ':4') \
     & (lineitem['l_quantity'] <= ':4 + 10') \
     & (part['p_size'] > 1) \
     & (part['p_size'] < 5) \
     & (lineitem['l_shipmode'].isin(('AIR', 'AIR REG'))) \
     & (lineitem['l_shipinstruct'] == 'DELIVER IN PERSON')
c1

p_k.p_partkey == l_k.l_partkey && p_k.p_brand == ":1" && (p_k.p_container == "SM CASE" || p_k.p_container == "SM BOX" || p_k.p_container == "SM PACK" || p_k.p_container == "SM PKG") && l_k.l_quantity >= ":4" && l_k.l_quantity <= ":4 + 10" && p_k.p_size > 1 && p_k.p_size < 5 && (l_k.l_shipmode == "AIR" || l_k.l_shipmode == "AIR REG") && l_k.l_shipinstruct == "DELIVER IN PERSON"

## Aggregation

### min()

### max()

# TPCH - 1 ✅

In [8]:
import pysdql

lineitem = pysdql.Relation(name='lineitem', cols=pysdql.LINEITEM_COLS)
lineitem = lineitem[lineitem['l_shipdate'] <= ':date']
lineitem.groupby(['l_returnflag', 'l_linestatus']).aggr(sum_qty=(lineitem['l_quantity'], 'sum'),
                                                        sum_base_price=(lineitem['l_extendedprice'], 'sum'),
                                                        sum_disc_price=(lineitem['l_extendedprice'] * (1 - lineitem['l_discount']),'sum'),
                                                        sum_charge=(lineitem['l_extendedprice'] * (1 - lineitem['l_discount']) * (1 + lineitem['l_tax']), 'sum'),
                                                        avg_qty=(lineitem['l_quantity'], 'avg'),
                                                        avg_price=(lineitem['l_extendedprice'], 'avg'),
                                                        avg_disc=(lineitem['l_discount'], 'avg'),
                                                        count_order=(lineitem['*'], 'count'))

let lmp = sum (<l_k, l_v> in lineitem) if (l_k.l_shipdate <= ":date") then { l_k } in
let tmpa = sum (<l_k, l_v> in lmp) { < l_returnflag=l_k.l_returnflag, l_linestatus=l_k.l_linestatus > -> { l_k -> l_v } } in
let tmp = sum (<ta_k, ta_v> in tmpa) { < l_returnflag=ta_k.l_returnflag, l_linestatus=ta_k.l_linestatus, group=ta_v > } in
let tmpb = sum (<t_k, t_v> in tmp) sum (<g_k, g_v> in t_k.group) { < l_returnflag=t_k.l_returnflag, l_linestatus=t_k.l_linestatus > -> < sum_qty=g_k.l_quantity * g_v, sum_base_price=g_k.l_extendedprice * g_v, sum_disc_price=(g_k.l_extendedprice * (1 - g_k.l_discount)) * g_v, sum_charge=((g_k.l_extendedprice * (1 - g_k.l_discount)) * (1 + g_k.l_tax)) * g_v, avg_qty_sum=g_k.l_quantity * g_v, avg_qty_count=g_v, avg_price_sum=g_k.l_extendedprice * g_v, avg_price_count=g_v, avg_disc_sum=g_k.l_discount * g_v, avg_disc_count=g_v, count_order=g_v > }
sum (<tb_k, tb_v> in tmpb) { < l_returnflag=tb_k.l_returnflag, l_linestatus=tb_k.l_linestatus, sum_qty=tb_v.sum_qty, 

agg_r

# TPCH - 2

In [9]:
import pysdql

part = pysdql.Relation(name='part', cols=pysdql.PART_COLS)
supplier = pysdql.Relation(name='supplier', cols=pysdql.SUPPLIER_COLS)
partsupp = pysdql.Relation(name='partsupp', cols=pysdql.PARTSUPP_COLS)
nation = pysdql.Relation(name='nation', cols=pysdql.NATION_COLS)
region = pysdql.Relation(name='region', cols=pysdql.REGION_COLS)

# TPCH - 3 ✅

In [10]:
import pysdql

customer = pysdql.Relation(name='customer', cols=pysdql.CUSTOMER_COLS)
orders = pysdql.Relation(name='orders', cols=pysdql.ORDERS_COLS)
lineitem = pysdql.Relation(name='lineitem', cols=pysdql.LINEITEM_COLS)

r = pysdql.merge(customer, orders, lineitem,
                 on=((customer['c_custkey'] == orders['o_custkey'])
                     & (orders['o_orderkey'] == lineitem['l_orderkey'])))

r = r[(customer['c_mktsegment'] == ':1') & (orders['o_orderdate'] < ':2') & (lineitem['o_orderdate'] > ':2')]

r = r.groupby(['l_orderkey', 'o_orderdate', 'o_shippriority'])\
     .aggr(revenue=((lineitem['l_extendedprice'] * (1 - lineitem['l_discount'])), 'sum'))

let R = sum (<c_k, c_v> in customer) sum (<o_k, o_v> in orders) sum (<l_k, l_v> in lineitem) if(c_k.c_custkey == o_k.o_custkey && o_k.o_orderkey == l_k.l_orderkey)
  { concat(concat(c_k, o_k), l_k) -> c_v * o_v * l_v } in
let rmp = sum (<r_k, r_v> in R) if (r_k.c_mktsegment == ":1" && r_k.o_orderdate < ":2" && r_k.o_orderdate > ":2") then { r_k } in
let tmpa = sum (<r_k, r_v> in rmp) { < l_orderkey=r_k.l_orderkey, o_orderdate=r_k.o_orderdate, o_shippriority=r_k.o_shippriority > -> { r_k -> r_v } } in
let tmp = sum (<ta_k, ta_v> in tmpa) { < l_orderkey=ta_k.l_orderkey, o_orderdate=ta_k.o_orderdate, o_shippriority=ta_k.o_shippriority, group=ta_v > } in
let tmpb = sum (<t_k, t_v> in tmp) sum (<g_k, g_v> in t_k.group) { < l_orderkey=t_k.l_orderkey, o_orderdate=t_k.o_orderdate, o_shippriority=t_k.o_shippriority > -> < revenue=(g_k.l_extendedprice * (1 - g_k.l_discount)) * g_v > }
sum (<tb_k, tb_v> in tmpb) { < l_orderkey=tb_k.l_orderkey, o_orderdate=tb_k.o_orderdate, o_shippriority=tb_k.o_s

# TPCH - 4 ✅

In [11]:
import pysdql

lineitem = pysdql.Relation(name='lineitem', cols=pysdql.LINEITEM_COLS)
orders = pysdql.Relation(name='orders', cols=pysdql.LINEITEM_COLS)

r = pysdql.merge(lineitem, orders,
                 on=(lineitem['l_orderkey'] == orders['o_orderkey'])
                 )[lineitem['l_commitdate'] < lineitem['l_receiptdate']]

orders = orders[(orders['o_orderdate'] >= ':1') & (orders['o_orderdate'] < ':1 + 3 month') & r.exists()]

orders = orders.groupby(['o_orderpriority']).aggr(order_count=('*', 'count'))

let R = sum (<l_k, l_v> in lineitem) sum (<o_k, o_v> in orders) if(l_k.l_orderkey == o_k.o_orderkey)
  { concat(l_k, o_k) -> l_v * o_v } in
let rmp = sum (<r_k, r_v> in R) if (r_k.l_commitdate < r_k.l_receiptdate) then { r_k } in
let omp = sum (<o_k, o_v> in orders) if (o_k.o_orderdate >= ":1" && o_k.o_orderdate < ":1 + 3 month" && (sum (<r_k, r_v> in rmp) r_v) > 0) then { o_k } in
let tmpa = sum (<o_k, o_v> in omp) { < o_orderpriority=o_k.o_orderpriority > -> { o_k -> o_v } } in
let tmp = sum (<ta_k, ta_v> in tmpa) { < o_orderpriority=ta_k.o_orderpriority, group=ta_v > } in
let tmpb = sum (<t_k, t_v> in tmp) sum (<g_k, g_v> in t_k.group) { < o_orderpriority=t_k.o_orderpriority > -> < order_count=g_v > }
sum (<tb_k, tb_v> in tmpb) { < o_orderpriority=tb_k.o_orderpriority, order_count=tb_v.order_count > }


# TPCH - 5 ✅

In [12]:
import pysdql

customer = pysdql.Relation(name='customer', cols=pysdql.CUSTOMER_COLS)
orders = pysdql.Relation(name='orders', cols=pysdql.ORDERS_COLS)
lineitem = pysdql.Relation(name='lineitem', cols=pysdql.LINEITEM_COLS)
supplier = pysdql.Relation(name='supplier', cols=pysdql.SUPPLIER_COLS)
nation = pysdql.Relation(name='nation', cols=pysdql.NATION_COLS)
region = pysdql.Relation(name='region', cols=pysdql.REGION_COLS)

r = pysdql.merge(customer, orders, lineitem, supplier, nation, region,
                 on=(lineitem['l_orderkey'] == orders['o_orderkey'])
                    & (lineitem['l_suppkey'] == supplier['s_suppkey'])
                    & (customer['c_nationkey'] == supplier['s_nationkey'])
                    & (supplier['s_nationkey'] == nation['n_nationkey'])
                    & (nation['n_regionkey'] == region['r_regionkey']))

r = r[(region['r_name'] == ':1')
      & (orders['o_orderdate'] >= ':2')
      & (orders['o_orderdate'] >= ':2 + 1 year')]

r = r.groupby(['n_name']).aggr(revenue=((lineitem['l_extendedprice'] * (1 - lineitem['l_discount'])), 'sum'))


let R = sum (<c_k, c_v> in customer) sum (<o_k, o_v> in orders) sum (<l_k, l_v> in lineitem) sum (<s_k, s_v> in supplier) sum (<n_k, n_v> in nation) sum (<r_k, r_v> in region) if(l_k.l_orderkey == o_k.o_orderkey && l_k.l_suppkey == s_k.s_suppkey && c_k.c_nationkey == s_k.s_nationkey && s_k.s_nationkey == n_k.n_nationkey && n_k.n_regionkey == r_k.r_regionkey)
  { concat(concat(concat(concat(concat(c_k, o_k), l_k), s_k), n_k), r_k) -> c_v * o_v * l_v * s_v * n_v * r_v } in
let rmp = sum (<r_k, r_v> in R) if (r_k.r_name == ":1" && r_k.o_orderdate >= ":2" && r_k.o_orderdate >= ":2 + 1 year") then { r_k } in
let tmpa = sum (<r_k, r_v> in rmp) { < n_name=r_k.n_name > -> { r_k -> r_v } } in
let tmp = sum (<ta_k, ta_v> in tmpa) { < n_name=ta_k.n_name, group=ta_v > } in
let tmpb = sum (<t_k, t_v> in tmp) sum (<g_k, g_v> in t_k.group) { < n_name=t_k.n_name > -> < revenue=(g_k.l_extendedprice * (1 - g_k.l_discount)) * g_v > }
sum (<tb_k, tb_v> in tmpb) { < n_name=tb_k.n_name, revenue=tb_v.revenue

# TPCH - 6 ✅

In [13]:
import pysdql

lineitem = pysdql.Relation(name='lineitem', cols=pysdql.LINEITEM_COLS)

lineitem = lineitem[(lineitem['l_shipdate'] >= ':1')
                    & (lineitem['l_shipdate'] < ':1 + 1 year')
                    & (lineitem['l_discount'] > ':2 - 0.01')
                    & (lineitem['l_discount'] < ':2 + 0.01')
                    & (lineitem['l_quantity'] > ':3')]

lineitem = lineitem.aggr(revenue=(lineitem['l_extendedprice'] * lineitem['l_discount'], 'sum'))

let lmp = sum (<l_k, l_v> in lineitem) if (l_k.l_shipdate >= ":1" && l_k.l_shipdate < ":1 + 1 year" && l_k.l_discount > ":2 - 0.01" && l_k.l_discount < ":2 + 0.01" && l_k.l_quantity > ":3") then { l_k } in
sum (<l_k, l_v> in lmp) < revenue=(l_k.l_extendedprice * l_k.l_discount) * l_v >


TPCH - 7

TPCH - 8

TPCH - 9

# TPCH - 10 ✅

In [14]:
import pysdql

customer = pysdql.Relation(name='customer', cols=pysdql.CUSTOMER_COLS)
orders = pysdql.Relation(name='orders', cols=pysdql.ORDERS_COLS)
lineitem = pysdql.Relation(name='lineitem', cols=pysdql.LINEITEM_COLS)
nation = pysdql.Relation(name='nation', cols=pysdql.NATION_COLS)

r = pysdql.merge(customer, orders, lineitem, nation,
                 on=(customer['c_custkey'] == orders['o_custkey'])
                    & (lineitem['l_orderkey'] == orders['o_custkey'])
                    & (customer['c_nationkey'] == nation['n_nationkey'])
                 )

r = r[(orders['o_orderdate'] >= ':1')
      & (orders['o_orderdate'] >= ':1 + 3 month')
      & (lineitem['l_returnflag'] == 'R')]

r = r.groupby(['c_custkey', 'c_name', 'c_acctbal', 'c_phone', 'n_name', 'c_address','c_comment'])\
     .aggr(revenue=((lineitem['l_extendedprice'] * (1 - lineitem['l_discount'])), 'sum'))

let R = sum (<c_k, c_v> in customer) sum (<o_k, o_v> in orders) sum (<l_k, l_v> in lineitem) sum (<n_k, n_v> in nation) if(c_k.c_custkey == o_k.o_custkey && l_k.l_orderkey == o_k.o_custkey && c_k.c_nationkey == n_k.n_nationkey)
  { concat(concat(concat(c_k, o_k), l_k), n_k) -> c_v * o_v * l_v * n_v } in
let rmp = sum (<r_k, r_v> in R) if (r_k.o_orderdate >= ":1" && r_k.o_orderdate >= ":1 + 3 month" && r_k.l_returnflag == "R") then { r_k } in
let tmpa = sum (<r_k, r_v> in rmp) { < c_custkey=r_k.c_custkey, c_name=r_k.c_name, c_acctbal=r_k.c_acctbal, c_phone=r_k.c_phone, n_name=r_k.n_name, c_address=r_k.c_address, c_comment=r_k.c_comment > -> { r_k -> r_v } } in
let tmp = sum (<ta_k, ta_v> in tmpa) { < c_custkey=ta_k.c_custkey, c_name=ta_k.c_name, c_acctbal=ta_k.c_acctbal, c_phone=ta_k.c_phone, n_name=ta_k.n_name, c_address=ta_k.c_address, c_comment=ta_k.c_comment, group=ta_v > } in
let tmpb = sum (<t_k, t_v> in tmp) sum (<g_k, g_v> in t_k.group) { < c_custkey=t_k.c_custkey, c_name=t_k.c_

# TPCH - 11

In [15]:
import pysdql

partsupp = pysdql.Relation(name='partsupp', cols=pysdql.PARTSUPP_COLS)
supplier = pysdql.Relation(name='supplier', cols=pysdql.SUPPLIER_COLS)
nation = pysdql.Relation(name='nation', cols=pysdql.NATION_COLS)

r = pysdql.merge(partsupp, supplier, nation,
                 on=((partsupp['ps_suppkey'] == supplier['s_suppkey'])
                     & (supplier['s_nationkey'] == nation['n_nationkey'])
                     )
                 )

r = r[(nation['n_name'] == ':1')]

r = r.groupby(['ps_partkey']).aggr(value=(partsupp['ps_supplycost'] * partsupp['ps_availqty'], 'sum'))

let R = sum (<p_k, p_v> in partsupp) sum (<s_k, s_v> in supplier) sum (<n_k, n_v> in nation) if(p_k.ps_suppkey == s_k.s_suppkey && s_k.s_nationkey == n_k.n_nationkey)
  { concat(concat(p_k, s_k), n_k) -> p_v * s_v * n_v } in
let rmp = sum (<r_k, r_v> in R) if (r_k.n_name == ":1") then { r_k } in
let tmpa = sum (<r_k, r_v> in rmp) { < ps_partkey=r_k.ps_partkey > -> { r_k -> r_v } } in
let tmp = sum (<ta_k, ta_v> in tmpa) { < ps_partkey=ta_k.ps_partkey, group=ta_v > } in
let tmpb = sum (<t_k, t_v> in tmp) sum (<g_k, g_v> in t_k.group) { < ps_partkey=t_k.ps_partkey > -> < value=(g_k.ps_supplycost * g_k.ps_availqty) * g_v > }
sum (<tb_k, tb_v> in tmpb) { < ps_partkey=tb_k.ps_partkey, value=tb_v.value > }


TPCH - 12

TPCH - 13

TPCH - 14

TPCH - 15

TPCH - 16

# TPCH - 17

In [16]:
lineitem = pysdql.Relation(name='lineitem', cols=pysdql.LINEITEM_COLS)
part = pysdql.Relation(name='part', cols=pysdql.PART_COLS)

part_agg = lineitem.groupby(['l_partkey']) \
    .aggr(agg_partkey=lineitem['l_partkey'], avg_quantity=(0.2 * lineitem['l_quantity'], 'avg')) \
    .rename('part_agg')

r = pysdql.merge(lineitem, part,
                 on=((part['p_partkey'] == lineitem['l_partkey'])
                     & (part_agg['agg_partkey'] == lineitem['l_partkey']))
                 )

r = r[(part['p_brand'] == ':1')
      & (part['p_container'] == ':2')
      & (lineitem['l_quantity'] < part_agg['avg_quantity'])]

r = r.aggr(avg_yearly=((lineitem['l_extendedprice'] / 7.0), 'sum'))

let tmp = sum (<l_k, l_v> in lineitem) { < l_partkey=l_k.l_partkey > -> { l_k -> l_v } } in
let lmp = sum (<t_k, t_v> in tmp) { < l_partkey=t_k.l_partkey, group=t_v > } in
let tmpa = sum (<l_k, l_v> in lmp) sum (<g_k, g_v> in l_k.group) { < l_partkey=l_k.l_partkey > -> < avg_quantity_sum=(0.2 * g_k.l_quantity) * g_v, avg_quantity_count=g_v > }
sum (<ta_k, ta_v> in tmpa) { < agg_partkey=ta_k.l_partkey, avg_quantity=(ta_v.avg_quantity_sum / ta_v.avg_quantity_count) > }
let R = sum (<l_k, l_v> in lineitem) sum (<p_k, p_v> in part) if(p_k.p_partkey == l_k.l_partkey && a_k.agg_partkey == l_k.l_partkey)
  { concat(l_k, p_k) -> l_v * p_v } in
let rmp = sum (<r_k, r_v> in R) if (r_k.p_brand == ":1" && r_k.p_container == ":2" && r_k.l_quantity < r_k.avg_quantity) then { r_k } in
sum (<r_k, r_v> in rmp) < avg_yearly=(l_k.l_extendedprice / 7.0) * r_v >


TPCH - 18

# TPCH - 19 ✅

In [17]:
import pysdql

lineitem = pysdql.Relation(name='lineitem', cols=pysdql.LINEITEM_COLS)
part = pysdql.Relation(name='part', cols=pysdql.PART_COLS)

c1 = (part['p_brand'] == ':1') \
     & (part['p_container'].isin(('SM CASE', 'SM BOX', 'SM PACK', 'SM PKG'))) \
     & (lineitem['l_quantity'] >= ':4') \
     & (lineitem['l_quantity'] <= ':4 + 10') \
     & (part['p_size'] > 1) \
     & (part['p_size'] < 5) \
     & (lineitem['l_shipmode'].isin(('AIR', 'AIR REG'))) \
     & (lineitem['l_shipinstruct'] == 'DELIVER IN PERSON')
c2 = (part['p_brand'] == ':2') \
     & (part['p_container'].isin(('MED BAG', 'MED BOX', 'MED PKG', 'MED PACK'))) \
     & (lineitem['l_quantity'] >= ':5') \
     & (lineitem['l_quantity'] <= ':5 + 10') \
     & (part['p_size'] > 1) \
     & (part['p_size'] < 10) \
     & (lineitem['l_shipmode'].isin(('AIR', 'AIR REG'))) \
     & (lineitem['l_shipinstruct'] == 'DELIVER IN PERSON')
c3 = (part['p_brand'] == ':3') \
     & (part['p_container'].isin(('LG CASE', 'LG BOX', 'LG PACK', 'LG PKG'))) \
     & (lineitem['l_quantity'] >= ':6') \
     & (lineitem['l_quantity'] <= ':6 + 10') \
     & (part['p_size'] > 1) \
     & (part['p_size'] < 15) \
     & (lineitem['l_shipmode'].isin(('AIR', 'AIR REG'))) \
     & (lineitem['l_shipinstruct'] == 'DELIVER IN PERSON')

r = pysdql.merge(lineitem, part, on=(part['p_partkey'] == lineitem['l_partkey']))

r = r[c1|c2|c3]

r = r.aggr(revenue=((lineitem['l_extendedprice'] * (1 - lineitem['l_discount'])), 'sum'))

let R = sum (<l_k, l_v> in lineitem) sum (<p_k, p_v> in part) if(p_k.p_partkey == l_k.l_partkey)
  { concat(l_k, p_k) -> l_v * p_v } in
let rmp = sum (<r_k, r_v> in R) if (r_k.p_brand == ":1" && (p_k.p_container == "SM CASE" || p_k.p_container == "SM BOX" || p_k.p_container == "SM PACK" || p_k.p_container == "SM PKG") && r_k.l_quantity >= ":4" && r_k.l_quantity <= ":4 + 10" && r_k.p_size > 1 && r_k.p_size < 5 && (l_k.l_shipmode == "AIR" || l_k.l_shipmode == "AIR REG") && r_k.l_shipinstruct == "DELIVER IN PERSON" || r_k.p_brand == ":2" && (p_k.p_container == "MED BAG" || p_k.p_container == "MED BOX" || p_k.p_container == "MED PKG" || p_k.p_container == "MED PACK") && r_k.l_quantity >= ":5" && r_k.l_quantity <= ":5 + 10" && r_k.p_size > 1 && r_k.p_size < 10 && (l_k.l_shipmode == "AIR" || l_k.l_shipmode == "AIR REG") && r_k.l_shipinstruct == "DELIVER IN PERSON" || r_k.p_brand == ":3" && (p_k.p_container == "LG CASE" || p_k.p_container == "LG BOX" || p_k.p_container == "LG PACK" || p_k.p_c

TPCH - 20

TPCH - 21

TPCH - 22